In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import os.path
import pandas as pd
import numpy as np

In [38]:
df_acoes = pd.read_csv('acoes-listadas-b3.csv')
todas_as_acoes = df_acoes['Ticker'].to_list()
todas_as_acoes_nparray = np.array(todas_as_acoes)
#selecione as 300 primeiras ações
lista_acoes2 = todas_as_acoes_nparray[0:300]


300

In [177]:
lista_acoes = np.array([])

len(lista_acoes)

0

In [174]:
servico = Service(ChromeDriverManager().install())
lista_acoes_validas = []
acoes_com_erro = []
navegador = webdriver.Chrome(service=servico)
navegador.maximize_window()
for acao in lista_acoes:


    navegador.get(f"https://investidor10.com.br/acoes/{acao}/")

    tabela_resultados = navegador.find_element(By.ID, "results_table")
    # desse a tela ate o elemento
    navegador.execute_script("arguments[0].scrollIntoView();", tabela_resultados)
    time.sleep(2)
    # verifica se ta visivil
    while not tabela_resultados.is_displayed():
        navegador.execute_script("arguments[0].scrollIntoView();", tabela_resultados)

    time.sleep(2)

    botoes = tabela_resultados.find_element(By.CLASS_NAME, "nav")
    botoes = botoes.find_elements(By.TAG_NAME, "li")


    bt_trimestral = botoes[3]
    bt_trimestral.click()
    time.sleep(1)

    dropdow = navegador.find_element(By.CLASS_NAME, "select2-results__options")
    dropdow = dropdow.find_elements(By.TAG_NAME, "li")
    dropdow[1].click()
    time.sleep(1)

    bt_valores = botoes[1]
    bt_valores.click()
    time.sleep(0.5)
    # clica no botao onde tem o texto "Valores Detalhados"
    dropdow = navegador.find_element(By.CLASS_NAME, "select2-results__options")
    dropdow = dropdow.find_elements(By.TAG_NAME, "li")
    dropdow[1].click()

    time.sleep(1)

    tabela = navegador.find_element(By.ID, "table-balance-results")

    linhas = tabela.find_elements(By.TAG_NAME, "tr")



    # A partir daqui, os dados estao sendo coletados
    # salve esses dados em um arquivo CSV
    colunas = linhas[0].find_elements(By.TAG_NAME, "th")
    if not os.path.isfile('dataset_ML.csv'): # Para nao escrever as colunas novamente
        with open('dataset_ML.csv', 'a', newline='', encoding='utf-8') as f:
            f.write("Empresa;Resultado;")
            for coluna in colunas:
                if coluna.text == "" or coluna.text == "#":
                    continue
                # inserindo uma condiçao para nao colocar o ";" na ultima coluna
                if coluna.text == "1T2019":
                    linha = coluna.text + "\n"
                else:
                    linha = coluna.text + ";"
                f.write(linha)
                print(coluna.text, end=" ")
            print()
            f.close()
    with open('dataset_ML.csv', 'a', newline='', encoding='utf-8') as f:
        indices_desejados = [1,4,6,9,13]
        for idx, linha in enumerate(linhas[1:], start=1):
            if idx in indices_desejados:
                colunas = linha.find_elements(By.TAG_NAME, "td")
                i = 1
                if len(colunas) !=60: #Significa que estão faltando trimestres nos dados da ação
                    acoes_com_erro.append(acao)
                    break #Passo para a próxima ação
                f.write(acao + ";")
                if acao not in lista_acoes_validas:
                    lista_acoes_validas.append(acao)
                for coluna in colunas:
                    
                    if coluna.text == "":
                        continue
                    #faça uma condicao que verifique coluna.text é o ultimo elemento e nao coloque o ";"
                    
                    if i == 21:
                        linha = coluna.text + "\n"
                    else:
                        linha = coluna.text + ";"
                    i += 1
                    f.write(linha)

                    print(coluna.text, end=" ")
        
                print()

lista_acoes = lista_acoes_validas

Receita Líquida - (R$) 133.247.000 126.296.000 138.491.000 177.390.000 155.410.000 216.090.000 221.180.000 210.140.000 264.950.000 235.040.000 227.000.000 231.240.000 281.680.000 226.760.000 78.020.000 145.030.000 146.053.000 145.110.000 152.174.000 146.000.000 
Lucro Líquido - (R$) 5.670.000 -9.635.000 10.383.000 29.570.000 36.210.000 28.440.000 4.730.000 -1.030.000 73.760.000 68.410.000 68.240.000 66.190.000 85.660.000 69.410.000 15.990.000 13.350.000 22.070.000 49.830.000 18.092.000 19.921.000 
EBIT - (R$) -6.041.000 -27.421.000 -813.000 32.730.000 15.410.000 19.830.000 -770.000 -6.110.000 79.490.000 53.370.000 75.990.000 74.360.000 23.580.000 76.460.000 19.400.000 13.340.000 15.130.000 49.680.000 20.090.000 22.660.000 
Dívida Líquida - (R$) -198.351.000 -94.848.000 -96.675.000 -134.521.000 -204.621.000 -155.452.000 -137.060.000 -134.400.000 -172.960.000 -215.120.000 -418.020.000 -366.580.000 -393.460.000 -198.260.000 -114.230.000 -118.080.000 -196.630.000 -179.300.000 -107.360.000 

In [175]:
lista_acoes_validas

['CRPG5', 'BEES4', 'BAHI3', 'NEXP3', 'EMAE4', 'MTSA4', 'TASA3']

In [176]:
acoes_com_erro

['MAPT4']

In [181]:
def create_pivot_table(df, index_col, columns_col, reindex_list):
    pivot_table = df.pivot_table(index=index_col, columns=columns_col, aggfunc='sum')
    pivot_table = pivot_table.reindex(reindex_list)
    pivot_table.columns = ['_'.join(col).strip() for col in pivot_table.columns.values] # Transformando o multi index da coluna em um index simples
    pivot_table = pivot_table.filter(like='23', axis=1) # Filtrando apenas os resultados do ano de 2023
    pivot_table = pivot_table.drop(pivot_table.filter(like='4T').columns, axis=1) # Removendo os resultados do 4T de 2023

    return pivot_table

"""Essa função cria uma tabela pivot a partir de um dataframe, onde index_col é a coluna que será utilizada como índice (Empresa), columns_col é a coluna que será utilizada como coluna (Resultado) e reindex_list é a lista que será utilizada para 
reindexar a tabela pivot (lista_acoes),pois a ordem original foi alterada. Eu estou colocando como índice cada empresa e como coluna os resultados de cada trimestre, filtrando pelo ano de 2023, que servirá como base de dados para análise"""

'Essa função cria uma tabela pivot a partir de um dataframe, onde index_col é a coluna que será utilizada como índice (Empresa), columns_col é a coluna que será utilizada como coluna (Resultado) e reindex_list é a lista que será utilizada para \nreindexar a tabela pivot (lista_acoes),pois a ordem original foi alterada. Eu estou colocando como índice cada empresa e como coluna os resultados de cada trimestre, filtrando pelo ano de 2023, que servirá como base de dados para análise'

In [194]:
def criacao_dataframe_dados_historicos(diretorio, lista_acoes):
    dataframes = []
    for acao in lista_acoes:
        df = pd.read_csv(f'{diretorio}/{acao} Dados Históricos.csv')
        #Inserindo o nome da empresa na coluna "Empresa" e colocando ela como index
        df['Empresa'] = acao
        df.set_index('Empresa', inplace=True)
        # Selecionando apenas o mes de novembro de 2023
        df = df[df['Data'] == '01.11.2023']
        # Calculando a variação - convertendo os valores em float
        df['Último'] = df['Último'].str.replace(',', '.').astype(float)
        df['Abertura'] = df['Abertura'].str.replace(',', '.').astype(float)
        df['Variação'] = (df['Último'] - df['Abertura']) / df['Abertura'] * 100
        # Criando a classificação, que vai servir como o Y do modelo
        if (df['Variação'] > 0).any():
            if (df['Variação'] > 10).any():
                df['Classificação'] = 2
            else:
                df['Classificação'] = 1
        else:
            df['Classificação'] = 0
        dataframes.append(df)
    df_final = pd.concat(dataframes)
    return df_final

"""Essa função está selecionando os arquivos csv dentro da pasta 'dados historicos' de acorco com a lista_acoes. Para cada arquivo csv, ela está inserindo o nome da empresa na coluna "Empresa" e colocando ela no começo do dataframe, 
selecionando apenas o mês de novembro de 2023, calculando a variação da ação nesse mês e criando a classificação, que vai servir como o Y do modelo. Se a empresa caiu, a classificação é 0, se subiu até 10%, a classificação é 1, 
e se subiu mais de 10%, a classificação é 2."""

'Essa função está selecionando os arquivos csv dentro da pasta \'dados historicos\' de acorco com a lista_acoes. Para cada arquivo csv, ela está inserindo o nome da empresa na coluna "Empresa" e colocando ela no começo do dataframe, \nselecionando apenas o mês de novembro de 2023, calculando a variação da ação nesse mês e criando a classificação, que vai servir como o Y do modelo. Se a empresa caiu, a classificação é 0, se subiu até 10%, a classificação é 1, \ne se subiu mais de 10%, a classificação é 2.'

In [8]:
def atribuir_classificacao(dataframe_resultados, dataframe_dados_historicos):
    dataframe_final = dataframe_resultados
    dataframe_final['Classificação'] = dataframe_dados_historicos['Classificação']
    return dataframe_final

# Essa função está atribuindo a coluna classificação do dataframe de dados históricos ao dataframe de resultados

Sobre as colunas do dataset:
- **Receita Líquida:** é a quantia de dinheiro que uma empresa ganha com suas vendas após a dedução de devoluções, descontos e impostos sobre vendas.
- **Lucro Líquido:** representa o lucro de uma empresa após todas as despesas e impostos terem sido deduzidos da receita total.
- **EBIT:** representa a lucratividade de uma empresa antes da consideração dos juros e impostos.
- **Dívida Líquida:** a diferença entre todos os compromissos financeiros pendentes e os recursos financeiros imediatamente disponíveis para liquidá-los.
- **ROE:** é uma medida que revela a capacidade de uma empresa gerar lucro a partir do dinheiro que os acionistas investiram nela.


In [185]:
# Ler o arquivo CSV
df = pd.read_csv('dataset_ML.csv', delimiter=';')
df2 = df.copy()
df2


,Empresa,Resultado,4T2023,3T2023,2T2023,1T2023,4T2022,3T2022,2T2022,1T2022,...,2T2021,1T2021,4T2020,3T2020,2T2020,1T2020,4T2019,3T2019,2T2019,1T2019
0,PETR4,Receita Líquida - (R$),134.258.000.000,124.828.000.000,113.840.000.000,139.068.000.000,158.579.000.000,170.076.000.000,170.960.000.000,141.641.000.000,...,110.710.000.000,86.174.000.000,74.972.000.000,70.730.000.000,50.898.000.000,75.470.000.000,72.627.978.000,77.051.002.000,72.566.996.000,79.999.000.000
1,PETR4,Lucro Líquido - (R$),31.163.000.000,26.760.000.000,28.936.000.000,38.307.000.000,43.502.000.000,46.236.000.000,54.484.000.000,44.783.000.000,...,43.041.000.000,1.276.000.000,60.452.000.000,-1.669.000.000,-2.813.000.000,-49.720.000.000,8.538.000.000,8.840.000.000,19.352.000.000,4.240.000.000
2,PETR4,EBIT - (R$),39.628.000.000,47.546.000.000,41.971.000.000,60.197.000.000,57.811.000.000,74.612.000.000,96.434.000.000,65.398.000.000,...,52.181.000.000,33.904.000.000,68.009.000.000,18.032.000.000,9.020.000.000,-45.440.000.000,14.135.000.000,14.300.000.000,39.240.000.000,14.520.000.000
3,PETR4,Dívida Líquida - (R$),227.799.000.000,238.304.000.000,218.390.000.000,204.125.000.000,224.510.000.000,256.715.000.000,180.367.000.000,189.848.000.000,...,266.429.000.000,332.856.000.000,328.268.000.000,373.509.000.000,390.005.000.000,380.190.000.000,-32.938.840.000,314.070.000.000,320.650.000.000,372.230.000.000
4,PETR4,ROE - (%),"8,16 %","6,90 %","7,76 %","9,49 %","11,89 %","12,37 %","13,23 %","10,26 %",...,"12,04 %","0,37 %","19,42 %","-0,62 %","-1,12 %","-20,93 %","2,76 %","2,97 %","6,21 %","1,43 %"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,TASA3,Receita Líquida - (R$),419.050.000,439.319.000,470.285.000,453.230.000,597.894.000,639.970.000,625.590.000,676.570.000,...,651.040.000,551.100.000,560.260.000,490.800.000,423.810.000,298.320.000,272.209.000,242.300.000,232.990.000,252.090.000
1156,TASA3,Lucro Líquido - (R$),42.530.000,26.001.000,48.863.000,35.400.000,121.033.000,103.140.000,100.820.000,194.990.000,...,193.630.000,68.100.000,279.460.000,102.240.000,39.040.000,-157.120.000,22.123.000,-26.372.000,43.630.000,4.050.000
1157,TASA3,EBIT - (R$),28.036.000,61.263.000,73.298.000,56.090.000,156.177.000,172.990.000,197.970.000,233.720.000,...,217.190.000,168.450.000,147.590.000,145.870.000,101.220.000,39.650.000,7.320.000,18.620.000,48.840.000,30.880.000
1158,TASA3,Dívida Líquida - (R$),324.638.000,360.285.000,243.914.000,94.178.000,181.462.000,238.873.000,-,-,...,-,-,-,-,-,-,-,-,-,-


In [192]:
#Obtendo os valores unicos da coluna 'Empresa' para atribuir para lista_acoes
lista_acoes = df2['Empresa'].unique()
dados = create_pivot_table(df2, 'Empresa', 'Resultado', lista_acoes)
# Removendo as linhas que possuem alguma celula com um '-'"dados historicos"
dados = dados.replace('-', np.nan)
dados = dados.dropna()
acoes_do_dataframe = np.array(dados.index)
dados


,1T2023_Dívida Líquida - (R$),1T2023_EBIT - (R$),1T2023_Lucro Líquido - (R$),1T2023_ROE - (%),1T2023_Receita Líquida - (R$),2T2023_Dívida Líquida - (R$),2T2023_EBIT - (R$),2T2023_Lucro Líquido - (R$),2T2023_ROE - (%),2T2023_Receita Líquida - (R$),3T2023_Dívida Líquida - (R$),3T2023_EBIT - (R$),3T2023_Lucro Líquido - (R$),3T2023_ROE - (%),3T2023_Receita Líquida - (R$)
Empresa,,,,,,,,,,,,,,,
PETR4,204.125.000.000,60.197.000.000,38.307.000.000,"9,49 %",139.068.000.000,218.390.000.000,41.971.000.000,28.936.000.000,"7,76 %",113.840.000.000,238.304.000.000,47.546.000.000,26.760.000.000,"6,90 %",124.828.000.000
MGLU3,5.038.335.000,16.328.000,-391.221.000,"-3,80 %",9.067.334.000,4.744.924.000,-35.923.000,-301.746.000,"-3,01 %",8.572.256.000,4.118.464.000,661.189.000,-498.330.000,"-5,23 %",8.578.818.000
OIBR3,20.956.798.000,-102.677.000,-1.267.087.000,"-5,47 %",2.535.792.000,21.232.781.000,-266.082.000,-844.524.000,"-3,53 %",2.454.171.000,22.719.925.000,-803.758.000,-2.830.492.000,"-10,55 %",2.421.776.000
B3SA3,-2.501.944.000,1.356.975.000,1.089.458.000,"5,30 %",2.460.480.000,-2.834.396.000,1.374.315.000,1.052.942.000,"5,13 %",2.477.122.000,-3.353.958.000,1.347.206.000,1.074.307.000,"5,31 %",2.490.082.000
COGN3,6.158.625.000,259.730.000,53.920.000,"0,45 %",1.329.876.000,6.161.440.000,147.387.000,-64.907.000,"-0,39 %",1.386.677.000,6.077.436.000,90.916.000,-116.656.000,"-0,85 %",1.270.040.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BAHI3,177.193.000,-7.610.000,-17.872.000,"-30,31 %",86.779.000,167.306.000,1.138.000,-14.127.000,"-22,00 %",100.934.000,178.597.000,-597.000,-15.005.000,"-45,80 %",94.225.000
NEXP3,48.337.000,-13.298.000,-17.843.000,"-18,42 %",30.066.000,50.187.000,26.464.000,29.871.000,"-23,20 %",37.420.000,50.369.000,4.650.000,4.411.000,"-4,09 %",-2.473.000
EMAE4,-447.375.000,1.038.000,18.882.000,"1,65 %",137.552.000,-468.834.000,10.281.000,37.848.000,"3,39 %",138.754.000,-457.016.000,39.526.000,41.102.000,"3,55 %",173.596.000


In [193]:
acoes_do_dataframe

array(['PETR4', 'MGLU3', 'OIBR3', 'B3SA3', 'COGN3', 'HAPV3', 'ITSA4',
       'ABEV3', 'PCAR3', 'SOMA3', 'BHIA3', 'CMIG4', 'RAIL3', 'CVCB3',
       'LREN3', 'CPLE6', 'AZUL4', 'PETR3', 'ALOS3', 'PETZ3', 'HBSA3',
       'PRIO3', 'MRVE3', 'POMO4', 'CPLE3', 'LWSA3', 'ELET3', 'CMIN3',
       'CCRO3', 'VAMO3', 'GGBR4', 'GOAU4', 'RAPT4', 'GOLL4', 'SUZB3',
       'RENT3', 'WEGE3', 'CRFB3', 'RRRP3', 'RADL3', 'RDOR3', 'BRFS3',
       'VBBR3', 'ANIM3', 'CSAN3', 'TIMS3', 'ECOR3', 'ENEV3', 'BRKM5',
       'STBP3', 'BEEF3', 'MRFG3', 'EQTL3', 'CSNA3', 'YDUQ3', 'QUAL3',
       'JBSS3', 'DXCO3', 'NTCO3', 'CEAB3', 'EMBR3', 'VIVA3', 'CYRE3',
       'GUAR3', 'ARZZ3', 'TOTS3', 'EGIE3', 'ALPA4', 'TEND3', 'UGPA3',
       'CPFE3', 'SLCE3', 'VIVT3', 'FLRY3', 'MOVI3', 'ODPV3', 'PDGR3',
       'NGRD3', 'SBSP3', 'EZTC3', 'JHSF3', 'AZEV4', 'ENJU3', 'CASH3',
       'DIRR3', 'MYPK3', 'FESA4', 'SRNA3', 'ELET6', 'BLAU3', 'IRBR3',
       'ENAT3', 'PSSA3', 'AERI3', 'GFSA3', 'SBFG3', 'CMIG3', 'SMTO3',
       'POSI3', 'GRN

In [198]:
df_final = criacao_dataframe_dados_historicos('dados historicos', acoes_do_dataframe)
df_final

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Variação,Classificação
Empresa,,,,,,,,,
PETR4,01.11.2023,35.91,33.73,"36,03","32,66","992,20M","3,37%",6.463089,1
MGLU3,01.11.2023,2.02,1.33,"2,35","1,31","4,53B","51,88%",51.879699,2
OIBR3,01.11.2023,0.65,0.60,"0,75","0,59","120,11M","10,17%",8.333333,1
B3SA3,01.11.2023,13.30,11.14,"13,45","11,13","729,75M","19,82%",19.389587,2
COGN3,01.11.2023,3.23,2.41,"3,26","2,39","528,64M","34,58%",34.024896,2
...,...,...,...,...,...,...,...,...,...
BAHI3,01.11.2023,8.06,9.15,"9,91","7,90","68,70K","-10,44%",-11.912568,0
NEXP3,01.11.2023,5.36,4.93,"5,70","4,91","24,60K","8,72%",8.722110,1
EMAE4,01.11.2023,59.51,59.39,"62,93","55,06","147,00K","0,02%",0.202054,1


In [199]:
df_classificado = atribuir_classificacao(dados, df_final)
df_classificado

,1T2023_Dívida Líquida - (R$),1T2023_EBIT - (R$),1T2023_Lucro Líquido - (R$),1T2023_ROE - (%),1T2023_Receita Líquida - (R$),2T2023_Dívida Líquida - (R$),2T2023_EBIT - (R$),2T2023_Lucro Líquido - (R$),2T2023_ROE - (%),2T2023_Receita Líquida - (R$),3T2023_Dívida Líquida - (R$),3T2023_EBIT - (R$),3T2023_Lucro Líquido - (R$),3T2023_ROE - (%),3T2023_Receita Líquida - (R$),Classificação
Empresa,,,,,,,,,,,,,,,,
PETR4,204.125.000.000,60.197.000.000,38.307.000.000,"9,49 %",139.068.000.000,218.390.000.000,41.971.000.000,28.936.000.000,"7,76 %",113.840.000.000,238.304.000.000,47.546.000.000,26.760.000.000,"6,90 %",124.828.000.000,1
MGLU3,5.038.335.000,16.328.000,-391.221.000,"-3,80 %",9.067.334.000,4.744.924.000,-35.923.000,-301.746.000,"-3,01 %",8.572.256.000,4.118.464.000,661.189.000,-498.330.000,"-5,23 %",8.578.818.000,2
OIBR3,20.956.798.000,-102.677.000,-1.267.087.000,"-5,47 %",2.535.792.000,21.232.781.000,-266.082.000,-844.524.000,"-3,53 %",2.454.171.000,22.719.925.000,-803.758.000,-2.830.492.000,"-10,55 %",2.421.776.000,1
B3SA3,-2.501.944.000,1.356.975.000,1.089.458.000,"5,30 %",2.460.480.000,-2.834.396.000,1.374.315.000,1.052.942.000,"5,13 %",2.477.122.000,-3.353.958.000,1.347.206.000,1.074.307.000,"5,31 %",2.490.082.000,2
COGN3,6.158.625.000,259.730.000,53.920.000,"0,45 %",1.329.876.000,6.161.440.000,147.387.000,-64.907.000,"-0,39 %",1.386.677.000,6.077.436.000,90.916.000,-116.656.000,"-0,85 %",1.270.040.000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BAHI3,177.193.000,-7.610.000,-17.872.000,"-30,31 %",86.779.000,167.306.000,1.138.000,-14.127.000,"-22,00 %",100.934.000,178.597.000,-597.000,-15.005.000,"-45,80 %",94.225.000,0
NEXP3,48.337.000,-13.298.000,-17.843.000,"-18,42 %",30.066.000,50.187.000,26.464.000,29.871.000,"-23,20 %",37.420.000,50.369.000,4.650.000,4.411.000,"-4,09 %",-2.473.000,1
EMAE4,-447.375.000,1.038.000,18.882.000,"1,65 %",137.552.000,-468.834.000,10.281.000,37.848.000,"3,39 %",138.754.000,-457.016.000,39.526.000,41.102.000,"3,55 %",173.596.000,1
